In [ ]:
!pip install autogluon autogluon==0.4.0

# Package Dependency

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

from autogluon.tabular import TabularDataset, TabularPredictor

import sklearn
from sklearn.utils import multiclass
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from itertools import cycle

In [ ]:
train = TabularDataset('/content/drive/MyDrive/Autoimmune_Disease/Independent Study/dataset/Coronna_Data_CERTAIN_classification_SC_0M_3M_bionaive TNF_all_Train.csv')
test = TabularDataset('/content/drive/MyDrive/Autoimmune_Disease/Independent Study/dataset/Coronna_Data_CERTAIN_classification_SC_0M_3M_bionaive TNF_all_Test.csv')

Loaded data from: /content/drive/MyDrive/Autoimmune_Disease/Independent Study/dataset/Coronna_Data_CERTAIN_classification_SC_0M_3M_bionaive TNF_all_Train.csv | Columns = 56 / 56 | Rows = 349 -> 349
Loaded data from: /content/drive/MyDrive/Autoimmune_Disease/Independent Study/dataset/Coronna_Data_CERTAIN_classification_SC_0M_3M_bionaive TNF_all_Test.csv | Columns = 56 / 56 | Rows = 86 -> 86


In [ ]:
train = train.drop(columns="DAS28_CRP_3M")
test = test.drop(columns="DAS28_CRP_3M")
train.head()

,grp,age,gender,final_education,race_grp,ethnicity,weight,BMI,height,newsmoker,...,md_global_assess,pt_global_assess,di,pt_pain,usresultsCRP,usresultsIgA,usresultsIgG,usresultsIgM,DAS28_CRP_0M,3MResponse
0,2.0,54.0,1.0,0.0,7.0,1.0,225.0,31.377703,71.0,2.0,...,50.0,25.0,0.285714,25.0,1.64,144.0,886.0,113.0,3.189429,Good
1,2.0,54.0,0.0,0.0,7.0,1.0,175.0,28.242654,66.0,2.0,...,35.0,17.0,0.250000,25.0,1.19,207.0,1010.0,56.0,2.935127,No Response
2,2.0,63.0,0.0,0.0,7.0,1.0,117.0,20.080811,64.0,0.0,...,40.0,10.0,0.000000,5.0,0.40,190.0,657.0,100.0,2.698947,No Response
3,2.0,61.0,0.0,0.0,7.0,1.0,240.0,39.933728,65.0,1.0,...,55.0,50.0,0.250000,60.0,1.00,44.0,948.0,51.0,3.719481,No Response
4,2.0,64.0,1.0,0.0,7.0,1.0,294.0,41.000198,71.0,1.0,...,55.0,55.0,0.285714,70.0,16.74,178.0,882.0,87.0,5.146565,Good


In [ ]:
print("Summary of occupation column: \n", train['3MResponse'].describe())

Summary of occupation column: 
 count          349
unique           3
top       Moderate
freq           133
Name: 3MResponse, dtype: object


In [ ]:
label = '3MResponse'
y_test = test[label]
test_data_nolabel = test.drop(columns=[label])

In [ ]:
save_path = '/content/drive/MyDrive/Autoimmune_Disease/Independent Study/autogluon_models/DAS28CRP_classificaiton'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train)

Beginning AutoGluon training ...
AutoGluon will save models to "/content/drive/MyDrive/Autoimmune_Disease/Independent Study/autogluon_models/DAS28CRP_classificaiton/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    349
Train Data Columns: 54
Label Column: 3MResponse
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	3 unique label values:  ['Good', 'No Response', 'Moderate']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11740.54 MB
	Train Data (Original)  Memory Usage: 0.15 MB (0.0% of available memory)
	Inferring data type of each feature based on colu

In [ ]:
predictor = TabularPredictor.load(save_path) 

y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.4186046511627907
Evaluations on test data:
{
    "accuracy": 0.4186046511627907,
    "balanced_accuracy": 0.4147722435078757,
    "mcc": 0.12543663190143045
}


Predictions:  
 0     No Response
1        Moderate
2     No Response
3        Moderate
4            Good
         ...     
81    No Response
82    No Response
83           Good
84           Good
85           Good
Name: 3MResponse, Length: 86, dtype: object


In [ ]:
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI,0.546512,0.442857,0.038393,0.026303,0.913160,0.038393,0.026303,0.913160,1,True,3
1,NeuralNetTorch,0.534884,0.428571,0.022001,0.016217,0.822874,0.022001,0.016217,0.822874,1,True,12
2,LightGBM,0.534884,0.428571,0.031503,0.005638,0.436730,0.031503,0.005638,0.436730,1,True,5
3,ExtraTreesGini,0.523256,0.428571,0.129278,0.102809,0.849501,0.129278,0.102809,0.849501,1,True,9
4,ExtraTreesEntr,0.511628,0.414286,0.126750,0.102822,0.844477,0.126750,0.102822,0.844477,1,True,10
5,RandomForestEntr,0.476744,0.428571,0.124627,0.103093,0.954889,0.124627,0.103093,0.954889,1,True,7
6,LightGBMXT,0.465116,0.457143,0.026949,0.005582,0.546416,0.026949,0.005582,0.546416,1,True,4
7,RandomForestGini,0.453488,0.371429,0.124583,0.103024,0.846548,0.124583,0.103024,0.846548,1,True,6
8,XGBoost,0.441860,0.428571,0.012670,0.006613,0.947729,0.012670,0.006613,0.947729,1,True,11
9,LightGBMLarge,0.430233,0.457143,0.011759,0.003214,1.423520,0.011759,0.003214,1.423520,1,True,13


In [ ]:
#confusion matrix
matrix = confusion_matrix(y_test, y_pred)
matrix = pd.DataFrame(matrix, columns = ['Moderate', 'Good', 'No Response'], index = ['Moderate', 'Good', 'No Response'])
matrix

,Moderate,Good,No Response
Moderate,16,5,8
Good,10,13,7
No Response,6,14,7
